In [1]:
# Load Validation Data:
import numpy as np 
import pandas as pd
validation_data = pd.read_csv('validation_data.csv')


/var/folders/r7/8vd8x98935g_j6xjscspbwr80000gn/T/ipykernel_98481/3787693704.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Creating a list of comments.

In [2]:
# Function to convert each sentence to the desired format
def convert_to_array(text):
    return f'{text},'

# Apply the function to the DataFrame
comments = validation_data['text'].apply(convert_to_array)
comments.describe()

count       500
unique      497
top       easy,
freq          2
Name: text, dtype: object

# 2. OpenAI API Labeling

In [3]:
!pip install --upgrade openai
!pip install transformers[torch]
!pip install accelerate -U
!pip install krippendorff
!pip install datasets

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 325.6 kB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near futu

In [4]:
# Import required Packages and Libraries
import os
import openai
from openai import OpenAIError
from openai import OpenAI

import pandas as pd
import time
import json
import numpy as np
import random
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import krippendorff
import torch
from transformers import TrainingArguments, Trainer, EvalPrediction, AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, IntervalStrategy
from datasets import Dataset, DatasetDict
from datetime import datetime

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
client = OpenAI(api_key = "sk-proj-nzxRWSLF2BlxuIZDtD50T3BlbkFJEbLOkIlSA9KlwIJJuCQz")

# Define a function to query OpenAI's models via the API
def ask_gpt(System_Prompt, User_Query, tokens, temp=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, model="gpt-4-turbo"):
    """Function that Queries OpenAI API"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": System_Prompt},
            {"role": "user", "content": User_Query}],
        max_tokens=tokens,
        temperature=temp,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )
    return response

In [6]:
# 3. Define your Prompts:

## System: Role, task, and format
System_Prompt = """
You are an AI expert trained to analyze social media comments for insights into fashion brand perceptions and emotional sentiments. Your task is to interpret each comment, even when the references to brands or sentiments are indirect, and classify the comment into applicable categories based on the information provided.

Brand perception categories:
1. Product Quality: Indicate if the comment suggests any perception about the quality of fashion products.
2. Reputation & Heritage: Determine if the comment reflects on the brand's history or reputation in the fashion industry.
3. Customer Service: Assess if there are any mentions or implications regarding customer service experiences with fashion brands.
4. Social Impact: Consider if the comment discusses the brand's role or actions in social issues.
5. Ethical Practices: Evaluate any mentions of ethical practices by the fashion brand.
6. Sustainability: Look for any discussions related to environmental sustainability of the brand.

Emotional sentiments to identify based on the presence of specific keywords:
- 'love', 'great', 'amazing', 'impressed', 'thrilled', 'excited'
- 'horrible', 'bad', 'disappointed', 'terrible', 'worse', 'hate'

Analyze the text, assign it to relevant brand perception categories, and determine the emotional sentiment based on the keywords or overall tone of the comment. If the comment does not contain explicit keywords, use your judgment to infer the sentiment from the context. Provide your findings in a structured format, listing both the identified brand perception categories and the emotional sentiment.
If the comment even slightly vaguely implies something categorize it into categories i provided. If it's a reference or a very short phrase, still categoprize it. Don't leave any comment unlabeled in terms of brand perceotion and emotional sentiment. If the comment doesn't pertain to anything, randomly assign labels.

For each comment, determine which brand perception categories and emotional sentiments apply. Provide your analysis in a structured format, with two columns: one for 'brand_labels' and one for 'emotion_labels'. List all applicable categories and sentiments in their respective columns, formatted as arrays. For example, a comment that is classified under both 'product quality' and 'reputation & heritage' for brand perception, and 'love' and 'admiration' for emotional sentiment, should be formatted as follows:

Brand_labels: ["product quality", "reputation & heritage"]
Emotion_labels: ["love", "admiration"]

Please ensure to apply this structured approach to all comments, making judgment calls as necessary based on even slight or indirect implications within the comments.
"""

## User: The query you want to send to the model
User_Query = comments[5]
display(comments)

0                                 opium founding father,
1                                yall trippin fit clean,
2                                  might destroy lonely,
3                                     alr show us women,
4                       bad think jeans ripped pull bad,
                             ...                        
495                                   wheres hood scarf,
496                       cus dubai thats nissan altima,
497                   park toyota dubai switch mercedes,
498    south africa everything would gone including p...
499                 thanks god thats love homeالحمدالله,
Name: text, Length: 500, dtype: object

In [7]:
response = ask_gpt(System_Prompt, User_Query, tokens=1000, temp=0, model="gpt-3.5-turbo")

# Example usage
display(response)

answer = response.choices[0].message.content
print(answer)

ChatCompletion(id='chatcmpl-9KzK40dBHwaFz1JPQWvJtX5U1KdMW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Brand_labels: ["Product Quality"]\nEmotion_labels: []', role='assistant', function_call=None, tool_calls=None))], created=1714789312, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=12, prompt_tokens=542, total_tokens=554))

Brand_labels: ["Product Quality"]
Emotion_labels: []


In [8]:
myfunction = {
  "type": "function",
  "function": {
    "name": "predict_label",
    "description": "Identify the brand perception category/categories and emotional sentiments based on the text",
    "parameters": {
      "type": "object",
      "properties": {
        "brand_labels": {
          "type": "array",
          "items": {
            "type": "string",
            "enum": [
              "product quality", 
              "reputation & heritage", 
              "customer service", 
              "social impact", 
              "ethical practices",
              "sustainability"
            ]
          },
          "description": "Brand perception labels for the social media comments."
        },
        "emotion_labels": {
          "type": "array",
          "items": {
            "type": "string",
            "enum": [
              "love", 
              "admiration", 
              "disgust", 
              "disapproval", 
              "excitement", 
              "optimism", 
              "disappointment", 
              "approval", 
              "pride"
            ]
          },
          "description": "Emotional sentiment labels for the social media comments."
        }
      },
      "required": [
        "brand_labels", 
        "emotion_labels"
      ]
    }
  }
}

In [9]:
def classify_gpt(text, model="gpt-3.5-turbo", tokens=200, temp=0.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
    """Function that uses OpenAI's API to classify text based on brand perception and emotional sentiment"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": System_Prompt},  # System prompt that explains the task to the model
                {"role": "user", "content": text}  # User input that needs to be classified
            ],
            max_tokens=tokens,
            temperature=temp,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty
        )
        # Correct way to access the response content and tokens
        content = response.choices[0].message.content  # Directly accessing the 'content' attribute of the 'message' object
        total_tokens = response.usage.total_tokens  # Directly accessing 'total_tokens' attribute of the 'usage' object
        return content, total_tokens
    except Exception as e:
        return f"Error: {str(e)}", 0

In [10]:
start_time = time.time()

# 7. Get all texts classified
results = []
tokens_used = 0
review_count = 0

# Query for each text the OpenAI model
for text in comments:

  # Error handling becomes important when you work with APIs. We imported OpenAIError to show errors and allow us to handle them
  try:
    response, tokens = classify_gpt(text, top_p=0.1, model="gpt-4-turbo")
    results.append((text, response, tokens))
    tokens_used += tokens
    review_count += 1
    if review_count % 50 == 0 and review_count != 0:
      print(f"Processed {review_count} comments") #Check to see the progress every 50 reviews
  except OpenAIError as e:
    # Handle all OpenAI API errors
    print(f"Error: {e}")

# Calculate and print the elapsed time
elapsed_time = time.time() - start_time
if elapsed_time > 60:
    print(f"Total time: {elapsed_time / 60:.2f} minutes")
else:
    print(f"Total time: {elapsed_time:.2f} seconds")

Processed 50 comments
Processed 100 comments
Processed 150 comments
Processed 200 comments
Processed 250 comments
Processed 300 comments
Processed 350 comments
Processed 400 comments
Processed 450 comments
Processed 500 comments
Total time: 9.38 minutes


In [11]:
print(results[0])

# 5. Create a dataframe from the results
df = pd.DataFrame(results, columns=['Text', 'labels','Tokens_used'])
df.head()
print(df['labels'])

('opium founding father,', 'Brand_labels: ["reputation & heritage"]\nEmotion_labels: []', 548)
0      Brand_labels: ["reputation & heritage"]\nEmoti...
1      Brand_labels: ["product quality"]\nEmotion_lab...
2      Brand_labels: ["reputation & heritage"]\nEmoti...
3      Brand_labels: ["reputation & heritage"]\nEmoti...
4      Brand_labels: ["product quality"]\nEmotion_lab...
                             ...                        
495    Brand_labels: ["customer service"]\nEmotion_la...
496    Brand_labels: ["customer service"]\nEmotion_la...
497    Brand_labels: ["reputation & heritage"]\nEmoti...
498    Brand_labels: ["reputation & heritage"]\nEmoti...
499    Brand_labels: ["reputation & heritage"]\nEmoti...
Name: labels, Length: 500, dtype: object


In [12]:
df.to_csv('validated_labeled_data.csv', index=False)

In [18]:
df = pd.read_csv('validated_labeled_data.csv')

# 3. Cleaning the labeled dataset

We decided to get rid of the emotion_labels because GPT couldn't categorize many comments into a set of emotions we provided. 

In [19]:
def parse_labels(label_string):
    brand_labels = []
    emotion_labels = []
    
    # Extract brand labels
    brand_start = label_string.find("Brand_labels: [") + len("Brand_labels: [")
    brand_end = label_string.find("]", brand_start)
    brand_labels_str = label_string[brand_start:brand_end]
    if brand_labels_str:
        brand_labels = [label.strip().strip('"') for label in brand_labels_str.split(",")]
    
    # Extract emotion labels
    emotion_start = label_string.find("Emotion_labels: [") + len("Emotion_labels: [")
    emotion_end = label_string.find("]", emotion_start)
    emotion_labels_str = label_string[emotion_start:emotion_end]
    if emotion_labels_str:
        emotion_labels = [label.strip().strip('"') for label in emotion_labels_str.split(",")]
    
    return brand_labels, emotion_labels

# Apply the parsing function to the labels column
df[['brand_label', 'emotion_label']] = df['labels'].apply(lambda x: pd.Series(parse_labels(x)))

In [20]:
df = df.drop(['labels', 'Tokens_used'], axis=1)

In [21]:
df.head()

,Text,brand_label,emotion_label
0,"opium founding father,",[reputation & heritage],[]
1,"yall trippin fit clean,",[product quality],[love]
2,"might destroy lonely,",[reputation & heritage],[horrible]
3,"alr show us women,",[reputation & heritage],[neutral]
4,"bad think jeans ripped pull bad,",[product quality],[bad]


In [16]:
# # Filter out rows with empty emotion labels
# df = df[df['emotion_label'].apply(lambda x: len(x) > 0)]

# # Save the filtered data
# df.head()

,Text,brand_label,emotion_label
1,"yall trippin fit clean,",[product quality],[love]
2,"might destroy lonely,",[reputation & heritage],[horrible]
3,"alr show us women,",[reputation & heritage],[neutral]
4,"bad think jeans ripped pull bad,",[product quality],[bad]
5,itssbo bro somehow made opium look normal exce...,[product quality],[neutral]


In [22]:
df.describe()

,Text,brand_label,emotion_label
count,500,500,500
unique,497,11,25
top,"easy,",[reputation & heritage],[]
freq,2,210,156


In [23]:
df.to_csv('validated_labeled_data_cleaned.csv', index=False)